In [3]:
using Pkg; Pkg.activate(".")

  Activating project at `D:\Documents\School Projects\MTH8408\Projet`


In [4]:
Pkg.instantiate()

   Installed NaNMath ───────────── v1.0.0
   Installed SetRounding ───────── v0.2.1
   Installed Preferences ───────── v1.2.4
   Installed RoundingEmulator ──── v0.2.1
   Installed FastRounding ──────── v0.2.0
   Installed RecipesPipeline ───── v0.5.1
   Installed StaticArrays ──────── v1.4.0
   Installed Plots ─────────────── v1.25.11
   Installed CRlibm_jll ────────── v1.0.1+0
   Installed ErrorfreeArithmetic ─ v0.5.2
   Installed Latexify ──────────── v0.15.12
   Installed IntervalArithmetic ── v0.20.3
   Installed CRlibm ────────────── v1.0.1
Precompiling project...
  ✓ SetRounding
  ✓ NaNMath
  ✓ Preferences
  ✓ ErrorfreeArithmetic
  ✓ RoundingEmulator
  ✓ FastRounding
  ✓ JLLWrappers
  ✓ Latexify
  ✓ CRlibm_jll
  ✓ OpenSSL_jll
  ✓ Ogg_jll
  ✓ Opus_jll
  ✓ EarCut_jll
  ✓ JpegTurbo_jll
  ✓ Xorg_libXau_jll
  ✓ Xorg_libXdmcp_jll
  ✓ Xorg_xtrans_jll
  ✓ libpng_jll
  ✓ libfdk_aac_jll
  ✓ x265_jll
  ✓ x264_jll
  ✓ Libmount_jll
  ✓ RecipesPipeline
  ✓ Graphite2_jll
  ✓ Zstd_jll
  ✓ Libic

In [112]:
Pkg.add("CSV"); Pkg.add("DataFrames"); Pkg.add("LsqFit"); Pkg.add("Plots")

   Resolving package versions...
  No Changes to `D:\Documents\School Projects\MTH8408\Projet\Project.toml`
  No Changes to `D:\Documents\School Projects\MTH8408\Projet\Manifest.toml`
   Resolving package versions...
  No Changes to `D:\Documents\School Projects\MTH8408\Projet\Project.toml`
  No Changes to `D:\Documents\School Projects\MTH8408\Projet\Manifest.toml`
   Resolving package versions...
  No Changes to `D:\Documents\School Projects\MTH8408\Projet\Project.toml`
  No Changes to `D:\Documents\School Projects\MTH8408\Projet\Manifest.toml`
   Resolving package versions...
  No Changes to `D:\Documents\School Projects\MTH8408\Projet\Project.toml`
  No Changes to `D:\Documents\School Projects\MTH8408\Projet\Manifest.toml`


In [102]:
using DataFrames, LsqFit, CSV, Statistics

## Exploration des données

In [14]:
data_folder = "Data/"
file_name   = "weatherstats_montreal_daily.csv"
df = DataFrame(CSV.File(data_folder * file_name))
first(df, 5)

,date,max_temperature,avg_hourly_temperature,avg_temperature,min_temperature
,Date,Float64,Float64,Float64,Float64
1,2022-03-31,10.1,4.85,5.3,0.5
2,2022-03-30,3.0,-1.26,-2.35,-7.7
3,2022-03-29,-0.6,-4.58,-4.0,-7.4
4,2022-03-28,-6.9,-9.75,-9.55,-12.2
5,2022-03-27,2.5,-1.65,-3.5,-9.5
6,2022-03-26,7.1,4.04,4.25,1.4
7,2022-03-25,9.0,4.46,5.4,1.8
8,2022-03-24,8.2,4.37,4.55,0.9
9,2022-03-23,4.7,0.55,-0.25,-5.2


In [31]:
"pop" in names(df)

false

In [25]:
nrow(df)

10000

In [30]:
ncol(df)

70

In [42]:
## Get the data
temp  = df.avg_temperature
jours = [i for i in 1:length(df.avg_temperature)]

10000-element Vector{Int64}:
     1
     2
     3
     4
     5
     6
     7
     8
     9
    10
    11
    12
    13
     ⋮
  9989
  9990
  9991
  9992
  9993
  9994
  9995
  9996
  9997
  9998
  9999
 10000

## Régression non linéaire 

In [86]:
## Premier modèle classique du papier
# t: array of independent variable
# p: array of model parameters
modele_simple(t, p) = p[1] .+ p[2]*t .+ p[3]*cos.(2*pi*t/365.25) .+ p[4]*sin.(2*pi*t/365.25) .+  p[5]*cos.(2*pi*t/(10.7*365.25)) .+  p[6]*sin.(2*pi*t/(10.7*365.25))

modele_simple (generic function with 1 method)

In [90]:
modele_harmonique(t, p) = p[1] .+ p[2]*t .+ p[3]*cos.(2*pi*t/365.25) .+ p[4]*sin.(2*pi*t/365.25) .+  p[5]*cos.(2*pi*t/(10.7*365.25)) .+  p[6]*sin.(2*pi*t/(10.7*365.25)) .+  p[7]*cos.(4*pi*t/(365.25)) .+  p[8]*sin.(4*pi*t/(365.25))

modele_harmonique (generic function with 1 method)

In [94]:
p0_S  = [2.6, 3.5, -20.0, -8.31, -0.197, 0.211]
fit_S = curve_fit(modele_simple, jours, temp, p0_S)

LsqFit.LsqFitResult{Vector{Float64}, Vector{Float64}, Matrix{Float64}, Vector{Float64}}([7.692148549433008, -6.568308514822362e-5, -5.668829648232002, -14.250878022537323, 0.2921050884184193, 0.11832982248027779], [-3.2287506359544653, 4.178823772816427, 5.588218989598888, 10.89950624903156, 4.612756225645327, -3.3719609870505503, -4.75457589731039, -4.135019635715937, 0.4367760244439086, -0.4791213530164085  …  2.741103340097009, -5.071473638767533, -7.251785369401198, -2.019899843837166, -7.685885704074355, -8.149812221764016, -3.211749277709931, -2.8717673411919047, -6.6299374491116, -8.036331184970356], [0.9999999999934013 0.9999999999502168 … 0.9999987076412624 0.0016077024860833202; 0.9999999999934013 2.0000000000471077 … 0.9999948305677253 0.0032154007919543773; … ; 0.9999999999934013 9998.999999999956 … -0.9332426053096522 -0.35924676698286817; 0.9999999999934013 9999.99999999998 … -0.9326638372475743 -0.36074667913889624], true, Float64[])

In [92]:
p0_H  = [2.6, 3.5, -20.0, -8.31, -0.197, 0.211, 1.0, 2.0]
fit_H = curve_fit(modele_harmonique, jours, temp, p0_H)

LsqFit.LsqFitResult{Vector{Float64}, Vector{Float64}, Matrix{Float64}, Vector{Float64}}([7.698429441857615, -6.624289856695777e-5, -5.663344240343394, -14.244929294301345, 0.2940904948960037, 0.1169074239590533, 0.4532455029627756, -0.7365648689539086], [-2.7872593879229015, 4.594301254927101, 5.977205776151072, 11.261556847931963, 4.947457108093483, -3.064990898670132, -4.475684779969398, -3.8845223552482913, 0.6585982843857942, -0.2862212839163565  …  3.3336076927908738, -4.4577602962956036, -6.617589095862655, -1.3659710600981316, -7.012998305265072, -7.458762661568367, -2.5033556259187706, -2.146868315635854, -5.889391423118777, -7.281015171523746], [0.9999999999934757 0.9999999999502168 … 0.9994082115903262 0.03439806059828307; 0.9999999999934757 2.0000000000471077 … 0.9976335468795511 0.06875540849591907; … ; 0.9999999999934757 9998.999999999956 … 0.00967621241304924 -0.9999531843596766; 0.9999999999934757 10000.0000000002 … 0.04406693633545649 -0.9990285807367247], true, Float64

### Affichage des paramètres trouvés

In [95]:
fit_S.param

6-element Vector{Float64}:
   7.692148549433008
  -6.568308514822362e-5
  -5.668829648232002
 -14.250878022537323
   0.2921050884184193
   0.11832982248027779

In [96]:
fit_H.param

8-element Vector{Float64}:
   7.698429441857615
  -6.624289856695777e-5
  -5.663344240343394
 -14.244929294301345
   0.2940904948960037
   0.1169074239590533
   0.4532455029627756
  -0.7365648689539086

In [104]:
sum(fit_S.resid.^2)

213308.66581277817

In [105]:
sum(fit_H.resid.^2)

209578.65082257235

### Estimate goodness of fit

In [107]:
# se = standard_error(fit_S)
cov = estimate_covar(fit_S)

6×6 Matrix{Float64}:
  0.00866214   -1.29183e-6    2.85544e-5  …  -0.00037037  -0.000630293
 -1.29183e-6    2.58341e-10  -9.35249e-9      9.43813e-8   1.76955e-8
  2.85544e-5   -9.35249e-9    0.0042822       2.87306e-5   1.77622e-5
 -6.48364e-5    3.66064e-9   -1.1475e-5      -1.78494e-5   2.82712e-5
 -0.00037037    9.43813e-8    2.87306e-5      0.00422105  -5.32003e-5
 -0.000630293   1.76955e-8    1.77622e-5  …  -5.32003e-5   0.00449482

In [108]:
cov = estimate_covar(fit_H)

8×8 Matrix{Float64}:
  0.00851305   -1.26962e-6    2.87084e-5  …  -2.11797e-5  -4.88232e-5
 -1.26962e-6    2.53899e-10  -9.30871e-9      6.81852e-9   7.37583e-9
  2.87084e-5   -9.30871e-9    0.00420878     -2.31501e-5  -4.54935e-5
 -6.37979e-5    3.6435e-9    -1.13853e-5      3.90959e-5  -9.95886e-6
 -0.000363602   9.26779e-8    2.8589e-5      -1.90172e-5  -2.29909e-5
 -0.000619365   1.73762e-8    1.74917e-5  …  -1.02235e-5   1.84439e-6
 -2.11797e-5    6.81852e-9   -2.31501e-5      0.0041968   -1.17372e-5
 -4.88232e-5    7.37583e-9   -4.54935e-5     -1.17372e-5   0.00419495

## Plots

In [110]:
pred_S = modele_simple(jours, fit_S.param)
pred_H = modele_harmonique(jours, fit_H.param)

10000-element Vector{Float64}:
  2.5127406120770983
  2.2443012549271018
  1.9772057761510717
  1.7115568479319636
  1.4474571080934828
  1.185009101329868
  0.9243152200306026
  0.6654776447517085
  0.40859828438579415
  0.15377871608364352
 -0.09887987501953235
 -0.34927675621950943
 -0.5973117077462468
  ⋮
 -1.0586684488535494
 -0.8133925854786691
 -0.566392307209126
 -0.3177602962956041
 -0.06758909586265538
  0.18402893990186842
  0.437001694734928
  0.6912373384316329
  0.9466443740812297
  1.2031316843641462
  1.4606085768812223
  1.7189848284762532

In [134]:
using Plots

gr()
scatter(df.date, temp, label="Température de Montréal", markersize=1, markercolor="blue")
p = plot!(df.date, [pred_S pred_H], title="Prédiction des Modèles pour Montréal", label=["Modèle Simple" "Modèle avec Harmoniques"], linecolor=["red" "cyan"])
xlabel!("Time t")
plot!(size=(1000,400))
savefig("Results/Two_Models.png")